In [1]:
# Tensorly library
import tensorly as tl
import numpy as np
from functools import reduce

In [220]:
"""
TENSOR TO TENSOR REGRESSION v1    
  Inputs:
    X - Input Tensor, shape (N, I_1, ..., I_{M1})
    Y - Output Tensor, shape (N, J_1, ..., J_{M2})
    rank - The rank of the solution weights matrix
    lambda_reg - how much l2 regularization to use
    return_factors - whether to include the weight matrix factors along with the weight matrix itself
    eps - the precision of our estimate
    
  Outputs:
    W - weight matrix tensor, shape (I_1, ..., I_{M1}, J_1, ..., J_{M2})
    factors - List of CP factors which forms the CP decomposition of W. Will only be returned if return_factors==True
"""
def tensor_to_tensor_regression_2(X, Y, rank, lambda_reg=0.0, return_factors=False, eps=1e-6):
    # Check that the N is consistent between X and Y tensors
    if X.shape[0] != Y.shape[0]:
        print("Wrong leading dimensions for tensors X and Y")
    
    # Number of examples (leading dimension in the tensor)
    N = X.shape[0]
    M1 = tl.ndim(X) - 1
    M2 = tl.ndim(Y) - 1
    
    # Initialize the u_r and v_r vectors which we will directly optimize
    Us = [np.random.random((X.shape[m1 + 1], rank))-0.5 for m1 in range(M1)]
    Vs = [np.random.random((Y.shape[m2 + 1], rank))-0.5 for m2 in range(M2)]
    
    # Y vector 
    Y_vec = tl.tensor_to_vec(Y)
    
    steps = 0
    prev_error = -1
    #errors = [] # TODO: Remove for production (only used for testing)
    # Keep optimizing until the error has converged
    while steps < 100:
        
        # Optimizing all the Us
        for m1 in range(M1):
            C_r_mats = []
            for r in range(rank):
                # Columns we will kronecker together
                cols_to_kron = []
                for m1_p in range(M1):
                    if m1_p != m1:
                        # r-th column of U_m1
                        cols_to_kron.append(Us[m1_p][:,r])
                for m2_p in range(M2):
                    cols_to_kron.append(Vs[m2_p][:,r])
                contractor_shape = tuple(x for i, x in enumerate(X.shape[1:]) if i != m1) + Y.shape[1:]
                contractor = tl.vec_to_tensor(tl.tenalg.kronecker(cols_to_kron), contractor_shape)
                X_modes = [i for i in range(1, M1 + 1) if i != m1 + 1]
                # Create the C_r tensor
                C_r = tl.tenalg.contract(X, X_modes, contractor, range(M1 - 1)) 
                C_r_mats.append(tl.unfold(C_r, 1))
            # This is the transpose of C
            C_mat_t = tl.concatenate(C_r_mats, axis=0)
            reg_mat = 0 # TODO: Remove
            reg_mat = None
            for m1_p in range(M1):
                if m1_p != m1:
                    to_dot = tl.transpose(Us[m1_p]) @ Us[m1_p] 
                    if reg_mat is None:
                        reg_mat = to_dot
                    else:
                        reg_mat = reg_mat @ to_dot
            for m2_p in range(M2):
                to_dot = tl.transpose(Vs[m2_p]) @ Vs[m2_p]
                if reg_mat is None:
                    reg_mat = to_dot
                else:
                    reg_mat = reg_mat @ to_dot 
            reg_mat = lambda_reg * tl.tenalg.kronecker([reg_mat, tl.eye(X.shape[m1 + 1])])
            U_vec = np.linalg.inv(C_mat_t @ tl.transpose(C_mat_t) + reg_mat) @ C_mat_t @ Y_vec
            # Finally update the U_m1 matrix
            Us[m1] = tl.vec_to_tensor(U_vec, Us[m1].shape)
                
        # Optimizing all the Vs    
        for m2 in range(M2):
            d_r_vecs = []
            for r in range(rank):
                # Columns we will kronecker together
                cols_to_kron = []
                for m1_p in range(M1):
                    # r-th column of U_m1
                    cols_to_kron.append(Us[m1_p][:,r])
                for m2_p in range(M2):
                    if m2 != m2_p:
                        cols_to_kron.append(Vs[m2_p][:,r])
                contractor_shape = X.shape[1:] + tuple(y for i, y in enumerate(Y.shape[1:]) if i != m2)
                contractor = tl.tenalg.kronecker(cols_to_kron).reshape(contractor_shape)
                # Create the C_r tensor
                d_r = tl.tensor_to_vec(tl.tenalg.contract(X, range(1, M1 + 1), contractor, range(M1)))
                d_r_vecs.append(d_r.reshape((d_r.shape[0], 1)))
            D_mat = tl.concatenate(d_r_vecs, axis=1)
            # Compute the regularization component
            reg_mat = 0 # TODO: Remove
            reg_mat = None
            for m1_p in range(M1):
                to_dot = tl.transpose(Us[m1_p]) @ Us[m1_p]
                if reg_mat is None:
                    reg_mat = to_dot
                else:
                    reg_mat = reg_mat @ to_dot
            for m2_p in range(M2):
                if m2_p != m2:
                    to_dot = tl.transpose(Vs[m2_p]) @ Vs[m2_p] 
                    if reg_mat is None:
                        reg_mat = to_dot
                    else:
                        reg_mat = reg_mat @ to_dot
            reg_mat *= lambda_reg
            # Get the correct Y_m2 matrix
            Y_m2 = tl.unfold(Y, m2 + 1)
            # Do the final update
            Vs[m2] = Y_m2 @ D_mat @ np.linalg.inv(tl.transpose(D_mat) @ D_mat + reg_mat)
        
        # Normalization Step
#         for r in range(rank):
#             tot_norm = 1
#             for m1 in range(M1):
#                 nrm = tl.norm(Us[m1][:,r], 2)
#                 tot_norm *= nrm
#                 Us[m1][:,r] /= nrm
#             for m2 in range(M2):
#                 nrm = tl.norm(Vs[m2][:,r], 2)
#                 tot_norm *= nrm
#                 Vs[m2][:,r] /= nrm
#             tot_norm = tot_norm ** (1 / (M1 + M2))
#             for m1 in range(M1):
#                 Us[m1][:,r] *= tot_norm
#             for m2 in range(M2):
#                 Vs[m2][:,r] *= tot_norm
#             print("\n")
#         print("-------")
        # Compute error here
        # Construct W
        weight_shape = X.shape[1:] + Y.shape[1:]
        W = tl.zeros(weight_shape)
        for r in range(rank):
            to_kron = []
            for m1 in range(M1):
                to_kron.append(Us[m1][:,r])
            for m2 in range(M2):
                to_kron.append(Vs[m2][:,r])
            W += tl.vec_to_tensor(tl.tenalg.kronecker(to_kron), weight_shape)
        Y_pred = tl.tenalg.contract(X, range(1, M1 + 1), W, range(M1))
        error = np.square(Y_pred - Y).mean() + lambda_reg * np.square(W).mean()
        
        # Determine if we have converged
        if prev_error >= 0 and abs(prev_error - error) < eps:
            print("Converged after", steps, "steps. Final Error:", error)
            break
        else:
            print("Step:", steps, "Error:", error)
        # Reset the previous error
        prev_error = error
        # Next step
        steps += 1
        
    return W
        
    

In [221]:
N = 11
X = np.random.random((N, 5, 7)) * 1
Y = np.zeros((N, 2, 3))
# Setup Y tensor with some dummy data
for n in range(N):
    x = X[n]
    Y[n] = np.array([[x[0, 0] + x[1, 1] , 2 * x[1, 0] - x[3, 2], (x[4, 5] + 1) ** 2],
                    [-x[1, 6] + 3 * x[1, 5] ,  - x[0, 6] - x[3, 3], (x[2, 5] + 2) ** 2]])
# Now fit it
W = tensor_to_tensor_regression_2(X, Y, 5, lambda_reg=10)
# We can verify our answer is correct with the following
print(X.shape)
Y_pred = tl.tenalg.contract(X, range(1, tl.ndim(X)), W, range(tl.ndim(X) - 1))
print(np.square(Y_pred - Y).mean()) # Should print the same as the final error

Step: 0 Error: 18.16275993069426
Step: 1 Error: 8.09102742455311
Step: 2 Error: 13.394514565302394
Step: 3 Error: 68.07745768519946
Step: 4 Error: 4.101118524355739
Step: 5 Error: 171.950752783646
Step: 6 Error: 13.88113686014119
Step: 7 Error: 7946.069778940578
Step: 8 Error: 46.92156033375949
Step: 9 Error: 3.5928504992692605
Step: 10 Error: 127.12702568455802
Step: 11 Error: 139.43685628765854
Step: 12 Error: 1289229.1316185007
Step: 13 Error: 7.276515607389161
Step: 14 Error: 11.704128450701049
Step: 15 Error: 22.371921035986784
Step: 16 Error: 338.1023565980306
Step: 17 Error: 320.07950683184794
Step: 18 Error: 163.4997733781796
Step: 19 Error: 12.058214167353796
Step: 20 Error: 8.528218405856506
Step: 21 Error: 550.1893422456793
Step: 22 Error: 13.600906615520042
Step: 23 Error: 29.702306992717
Step: 24 Error: 8.09158156871639
Step: 25 Error: 42.954551589193684
Step: 26 Error: 7.381048879196067
Step: 27 Error: 7.539866427068356
Step: 28 Error: 13.085139081550816
Step: 29 Error: 2

In [156]:
# For timing
import time

start_time = time.time()
W = tensor_to_tensor_regression_2(X, Y, 3, lambda_reg=10)
print(time.time() - start_time)

Step: 0 Error: 2.1202665140625734
Step: 1 Error: 1.1475246083125783
Step: 2 Error: 0.6778765144120024
Step: 3 Error: 0.86160570805602
Step: 4 Error: 2.967933629566618
Step: 5 Error: 1.284519887069989
Step: 6 Error: 2.829593085627063
Step: 7 Error: 4.143177412073496
Step: 8 Error: 1.9275961320727732
Step: 9 Error: 2.6127807275000796
Step: 10 Error: 1.2684087549879925
Step: 11 Error: 0.7475411507168298
Step: 12 Error: 1.2581483351679648
Step: 13 Error: 1.138243391134839
Step: 14 Error: 1.1051371668056194
Step: 15 Error: 3.4863792514193053
Step: 16 Error: 1.4131749332234727
Step: 17 Error: 2.12466132070165
Step: 18 Error: 0.802233048253556
Step: 19 Error: 1.7191348388189736
Step: 20 Error: 2.056671509889354
Step: 21 Error: 1.041077093760336
Step: 22 Error: 0.6813546797543508
Step: 23 Error: 3.9765385522504957
Step: 24 Error: 2.8978430024059962
Step: 25 Error: 2.147042823074191
Step: 26 Error: 0.678284034758313
Step: 27 Error: 2.6581610830005586
Step: 28 Error: 1.7814105261381614
Step: 29 

In [3]:
tuple(x for i, x in enumerate((9,5,3,3,2,1)) if i != 1)

(9, 3, 3, 2, 1)

In [6]:
arr = [i for i in range(5) if i != 2]
print(arr)
arr.pop(1)
print(arr)

[0, 1, 3, 4]
[0, 3, 4]


In [202]:
a = np.array([[1,2], [3, 4]])
b = np.array([[5, 6], [7, 8]])
a @ b

array([[19, 22],
       [43, 50]])

In [167]:
cols_to_kron = [np.array([2, 3, 5]), np.array([7, 11, 13]), np.array([17, 19, 23])]
kron = tl.tenalg.kronecker(cols_to_kron)
arr = tl.vec_to_tensor(kron, (3, 3, 3))
print(arr[0, 1, 1])

418
